# <span style='color:#3b748a'>The rental data for Chicago is TOO large to upload to GitHub.</span>

<img src="../images/bikes_banner.jpg" width="1000" />

# <span style="color:#37535e">Bicycle Share Usage</span>

##  <span style='color:#3b748a'>Cleaning Chicago Divvy data</span>

<span style='color:#4095b5'>This notebook loads and cleans 12 months (July 2017 - June 2018) of data from the Chicago Divvy bicycle share. There is data going back to 2013 that could be cleaned and used. The quarterly data from Jul-Sep 2017 is in a very large csv file, so these months are broken out to be processed more easily.</span>

<span style='color:#4095b5'>Each row (observation) of data describes one bike ride on which a bike is taken. Each rental includes a starting place and time, a ending place and time, as well as duration, user, and bike information. </span>

## <span style='color:#3b748a'>Table of contents</span>
* <span style='color:#4095b5'>I.  <a href="#checking"><span style='color:#4095b5'>Data checking functions.</span></a></span>
* <span style='color:#4095b5'>II. <a href="#cleaning"><span style='color:#4095b5'>Data cleaning functions.</span></a></span>
* <span style='color:#4095b5'>III. <a href="#convert"><span style='color:#4095b5'>Extract Jul, Aug, Sep from 2017 Q3 data.</span></a></span>
* <span style='color:#4095b5'>IV. <a href="#import"><span style='color:#4095b5'>Import all data.</span></a></span>
* <span style='color:#4095b5'>V. <a href="#clean"><span style='color:#4095b5'>Clean all data.</span></a></span>
* <span style='color:#4095b5'>VI. <a href="#merge"><span style='color:#4095b5'>Merge the dataframes into 1 big one.</span></a></span>
* <span style='color:#4095b5'>VII. <a href="#explore"><span style='color:#4095b5'>Explore the data.</span></a></span>
* <span style='color:#4095b5'>VIII. <a href="#write"><span style='color:#4095b5'>Write the full DataFrame to a csv file.</span></a></span>

## <span style='color:#3b748a'>External data required</span>
<ul>
    <li><span style='color:#4095b5'>../data/chi/Divvy_Trips_XXX.csv for each quarter or month; NOT available in GitHub</span></li>
    <li><span style='color:#4095b5'>../data/chi/Divvy_Stations_2017_Q3Q4.csv for hub data; NOT available in GitHub</span></li>
</ul>

## <span style='color:#3b748a'>Links</span>
<ul>
    <li><a href="https://www.divvybikes.com/system-data"><span style='color:#4095b5'>Chicago Divvy data</span></a></li>
    <li><a href="plot_chi.ipynb"><span style='color:#4095b5'>Plotting Chicago data.</span></a></li>
</ul>
<hr>

In [1]:
# Let's get the administrative stuff done first
# import all the libraries and set up the plotting

import pandas as pd
import numpy as np
from datetime import datetime,timedelta
from geopy.distance import vincenty

# Gloabal variables to track 
trivial_duration = 0
trivial_distance = 0
outliers_latlon = 0
outliers_duration = 0
outliers_distance = 0

# GnBu_d
colors = ['#37535e', '#3b748a', '#4095b5', '#52aec9', '#72bfc4', '#93d0bf']

<hr>
<a name="checking"> </a>
## <span style='color:#3b748a'>I. Data checking functions</span>

In [2]:
# Check which non-numeric columns are missing values and what the possible values are for each object column

def check_cols(df):
    cols = df.select_dtypes([np.object]).columns
    for col in cols:
        print("{} is {} and values are {}.".format(col,df[col].dtype,df[col].unique()))
        n_nan = df[col].isnull().sum()
        if n_nan > 0:
            print("{} has {} NaNs".format(col,n_nan))
            
    cols = df.select_dtypes([np.int64,np.float64,np.uint64]).columns
    for col in cols:
        print("{} is {} and values are {} to {}.".format(col,df[col].dtype,df[col].min(),df[col].max()))
        n_nan = df[col].isnull().sum()
        if n_nan > 0:
            print("{} has {} NaNs".format(col,n_nan))
    return

In [3]:
# Check which numeric columns are missing values

def check_data(df):
    s = df.shape

    # Check for null values
    null_data = df.isnull().sum()
    null_data_count = sum(df.isnull().sum())
    print("Rows: {}\t Cols: {}\t NaNs: {}".format(s[0],s[1],null_data_count))
    if  null_data_count > 0:
        print("Columns with NaN: {}".format(list(null_data[null_data > 0].index)))

    return

<hr>
<a name="cleaning"></a>
## <span style='color:#3b748a'> II. Data cleaning functions</span>

<span style='color:#4095b5'>These functions clean the trip data.</span>

### <span style='color:#4095b5'>Drop columns *NOT* in Atlanta data.</span>
<span style='color:#52aec9'>I might want to add some back at some point.</span>

In [4]:
def drop_columns(df):
    cols_drop = ['trip_id', 'from_station_id', 'to_station_id','usertype', 'gender', 'birthyear',
                 '01 - Rental Details Rental ID', '03 - Rental Start Station ID', '02 - Rental End Station ID', 
                   'User Type', 'Member Gender', '05 - Member Details Member Birthday Year']

    # Can't drop a column that isn't there
    cols_drop = list(set(df.columns) & set(cols_drop))
    df.drop(cols_drop, axis=1, inplace=True)

    return df

### <span style='color:#4095b5'>Rename columns to match Atlanta data names.</span>

In [5]:
def rename_columns(df):
    df.rename(columns={'from_station_name' : 'Start Hub', 
                       'start_time' : 'Start Time', 
                       'to_station_name' : 'End Hub', 
                       'end_time' : 'End Time', 
                       'bikeid' :'Bike Name',
                       'tripduration' : 'Duration',
                       '03 - Rental Start Station Name'  : 'Start Hub', 
                       '01 - Rental Details Local Start Time' : 'Start Time', 
                       '02 - Rental End Station Name' : 'End Hub',
                       '01 - Rental Details Local End Time' : 'End Time', 
                       '01 - Rental Details Bike ID' :'Bike Name',
                       '01 - Rental Details Duration In Seconds Uncapped': 'Duration'        
                  }, inplace=True)
    return df

### <span style='color:#4095b5'>Merge with hub data.</span>
<span style='color:#52aec9'>We may have to use the start/end hubs to get start/end lat/long.</span>

In [6]:
def calc_latlong(df, df_hubs):
    df = df.merge(df_hubs, left_on='Start Hub', right_on='name', how='left')
    df.drop('name', axis = 1, inplace=True)
    df.rename(columns={'latitude' : 'Start Latitude', 
                       'longitude' : 'Start Longitude'
                      }, inplace=True)
    
    df = df.merge(df_hubs, left_on='End Hub', right_on='name', how='left')
    df.drop('name', axis = 1, inplace=True)
    df.rename(columns={'latitude' : 'End Latitude', 
                       'longitude' : 'End Longitude'
                      }, inplace=True)

    return df

### <span style='color:#4095b5'>Drop rows with nulls.</span>
<span style='color:#52aec9'>Some hubs do not have lat/long information. We need to figure out how to compute these if we are going to use them in a model!!</span>

In [7]:
def drop_nans(df):
    rows = df.shape[0]
    df.dropna(subset=['Start Latitude', 'Start Longitude', 
                      'End Latitude', 'End Longitude'], 
              inplace=True)
    print("Drop lat/lon rows: {}".format(rows-df.shape[0]))
    return df


### <span style='color:#4095b5'>Use appropriate datatypes.</span>
<span style='color:#52aec9'>For example, fix Date/Time objects and cast Latitude and Longitude to floats.</span>

In [8]:
def clean_datatypes(df):
    df['Start Latitude'] = df['Start Latitude'].astype(float)
    df['Start Longitude'] = df['Start Longitude'].astype(float)
    df['End Latitude'] = df['End Latitude'].astype(float)
    df['End Longitude'] = df['End Longitude'].astype(float)

    # Turn times in datetime
    df['Start Time'] = pd.to_datetime(df['Start Time'])
    df['End Time'] = pd.to_datetime(df['End Time'])

    # CREATE dates in datetime
    df['Start Date'] = df['Start Time'].dt.date
    df['End Date'] = df['End Time'].dt.date

    # Fix the durations
    if df['Duration'].dtype == np.object:
        df['Duration'] = df['Duration'].map(lambda cell: cell.replace(',',''))
    df['Duration'] = df['Duration'].astype(float)
    df['Duration'] = pd.to_timedelta(df['Duration'], unit='s')
    
    return df

### <span style='color:#4095b5'>Calculate distances.</span>
<span style='color:#52aec9'>Poor approximation. If bike was checked-out and returned to same station, will be trivial distance.</span>

In [9]:
def distance_calc (row):
    start = (row['Start Latitude'], row['Start Longitude'])
    stop = (row['End Latitude'], row['End Longitude'])

    return vincenty(start, stop).miles

In [10]:
def calc_distances(df):
    df['Distance [Miles]'] = df.apply (lambda row: distance_calc (row),axis=1)
    return df

### <span style='color:#4095b5'>Reorder columns.</span>
<span style='color:#52aec9'>Make order same as Atlanta data.</span>

In [11]:
def reorder_cols(df):
    columns = ['Start Hub', 'Start Latitude', 'Start Longitude', 'Start Date',
       'Start Time', 'End Hub', 'End Latitude', 'End Longitude', 'End Date',
       'End Time', 'Bike Name', 'Distance [Miles]', 'Duration']

    df = df.reindex(columns=columns)
    return df

### <span style='color:#4095b5'>Drop trivial trips.</span>
<span style='color:#52aec9'>Trivial trips have time less than 3 mins. We cannot drop for trivial distance, since we compute distance.</span>

In [12]:
def drop_trivial_trips_distance(df):
    df = df[df["Distance [Miles]"] > 0.02].copy()
    return df

In [13]:
def drop_trivial_trips_duration(df):
    df = df[df["Duration"] >= pd.to_timedelta('00:03:00')].copy()
    return df

In [14]:
def drop_trivial_trips(df):
    global trivial_duration
    global trivial_distance

    rows = df.shape[0]
    df = drop_trivial_trips_duration(df)
    rows_duration = df.shape[0]
    trivial_duration += rows-rows_duration

    # Calculated distance, don't drop
    # df = drop_trivial_trips_distance(df)
    rows_distance = df.shape[0]
    trivial_distance += rows_duration-rows_distance

    return df

### <span style='color:#4095b5'>Drop outliers.</span>
<ul>
    <li><span style='color:#52aec9'>Only use trips near Chicago.</span></li> 
    <li><span style='color:#52aec9'>Only use trips no longer than 24 hours.</span></li> 
</ul>

In [15]:
def drop_outliers_latlon(df):
#     df = df[df["Start Latitude"] < 33.9].copy()
#     df = df[df["End Latitude"] < 33.9].copy()
#     df = df[df["Start Latitude"] > 33.5].copy()
#     df = df[df["End Latitude"] > 33.5].copy()

#     df = df[df["Start Longitude"] < -83.0].copy()
#     df = df[df["End Longitude"] < -83.0].copy()

    return df

In [16]:
def drop_outliers_duration(df):
    df = df[df["Duration"] <= pd.to_timedelta('24:00:00')].copy()
    return df

In [17]:
def drop_outliers_distance(df):
    df_temp = df[df["Distance [Miles]"] >= 100.0]
    if df_temp.shape[0]:
        print("Long trip: ", df_temp[['Start Latitude','Start Longitude', 'Start Time', 
                                     'End Latitude', 'End Longitude', 'End Time', 
                                     'Distance [Miles]', 'Duration']])
    df = df[df["Distance [Miles]"] < 100.0].copy()
    return df

In [18]:
def drop_outliers(df):
    global outliers_latlon
    global outliers_duration
    global outliers_distance
    
    rows = df.shape[0]
    df = drop_outliers_latlon(df)
    rows_latlon = df.shape[0]
    outliers_latlon += rows - rows_latlon
    
    df = drop_outliers_duration(df)
    rows_duration = df.shape[0]
    outliers_duration += rows_latlon - rows_duration
    
    df = drop_outliers_distance(df)
    rows_distance = df.shape[0]
    outliers_distance += rows_duration - rows_distance
    
    return df

### <span style='color:#4095b5'>Pull all of the cleaning together.</span>

In [19]:
def clean_df(df, df_hubs=None):
    global trivial_duration
    global trivial_distance
    global outliers_latlon
    global outliers_duration
    global outliers_distance

    df = drop_columns(df)
    df = rename_columns(df)
    df = calc_latlong(df, df_hubs)
    df = drop_nans(df)
    df = clean_datatypes(df)
    df = calc_distances(df)
    df = reorder_cols(df)
    df = drop_trivial_trips(df)
    df = drop_outliers(df)

    # Information about rows dropped
    print("Trivial dur: {} dist: {}".format(trivial_duration, 
                                                                              trivial_distance))
    print("Outlier loc: {} dur: {} dist: {}".format(outliers_latlon,
                                                     outliers_duration,
                                                     outliers_distance))
    return df

<hr>
<a name="convert"></a>
## <span style='color:#3b748a'> III. Extract July, Aug, Sep 2017 from 2017 Q3 data.</span>
<ul>
    <li><span style='color:#4095b5'>Trip data is quarterly.</span></li>
    <li><span style='color:#4095b5'>The file is too huge to easily use.</span></li>
</ul>

In [20]:
if False:
    d = "2017_Q3"
    df = pd.read_csv("../data/chi/Divvy_Trips_"+str(d)+".csv")

    # Turn times in datetime
    df['Start Time'] = pd.to_datetime(df['start_time'])
    df['End Time'] = pd.to_datetime(df['end_time'])

    # 2017Q3 = 2017-07-01, 2017-08-01, 2017-09-01
    df7 = df[df['Start Time'] < datetime.strptime('2017-08-01 00:00:00', '%Y-%m-%d %H:%M:%S')].copy()
    df7.drop(['Start Time', 'End Time'], axis=1, inplace=True)
    print(df7.shape)
    df7.to_csv('../data/chi/Divvy_Trips_2017_07.csv', index=False)
    
    # 2017Q3 = 2017-07-01, 2017-08-01, 2017-09-01
    df8 = df[df['Start Time'] < datetime.strptime('2017-09-01 00:00:00', '%Y-%m-%d %H:%M:%S')].copy()
    df8 = df8[df8['Start Time'] >= datetime.strptime('2017-08-01 00:00:00', '%Y-%m-%d %H:%M:%S')].copy()
    df8.drop(['Start Time', 'End Time'], axis=1, inplace=True)
    print(df8.shape)
    df8.to_csv('../data/chi/Divvy_Trips_2017_08.csv', index=False)
    
    # 2017Q3 = 2017-07-01, 2017-08-01, 2017-09-01
    df9 = df[df['Start Time'] >= datetime.strptime('2017-09-01 00:00:00', '%Y-%m-%d %H:%M:%S')].copy()
    df9.drop(['Start Time', 'End Time'], axis=1, inplace=True)
    print(df9.shape)
    df9.to_csv('../data/chi/Divvy_Trips_2017_09.csv', index=False)

<hr>
<a name="import"></a>
## <span style='color:#3b748a'> IV. Import all data from Chicago.</span>
<ul>
    <li><span style='color:#4095b5'>Trip data is quarterly.</span></li>
    <li><span style='color:#4095b5'>Need hub data to calculate latitude/longitude.</span></li>
</ul>

In [21]:
# Chicago data is quartlerly, except for Sep 2017
trip_data = ["2017_07", "2017_08",
             "2017_09", "2017_Q4",
             "2018_Q1", "2018_Q2"]

In [22]:
# Dictionary of DataFrames, one for each quarter
df_data = dict()
for d in trip_data:
    df_data[d] = pd.read_csv("../data/chi/Divvy_Trips_"+str(d)+".csv")

In [23]:
# We need the hubs in order to lookup latitude/longitude
df_hubs = pd.read_csv("../data/chi/Divvy_Stations_2017_Q3Q4.csv")
df_hubs.drop(['id','city', 'dpcapacity', 'online_date'], axis=1, inplace=True)

<hr>
<a name="clean"></a>

## <span style='color:#3b748a'>V. Clean all data from Chicago.</span>
<ul>
    <li><span style='color:#4095b5'>Reform the data to match Atlanta data.</span></li>
    <li><span style='color:#4095b5'>Drop the trivial trips.</span></li>
    <li><span style='color:#4095b5'>Drop the outliers.</span></li>
    <li><span style='color:#4095b5'>Use appropriate coumn types.</span></li>
</ul>

In [24]:
# For each quarter, clean the DataFrame
print("Cleaning the data:")
for d in trip_data:
    print("Month: {} \nRows: {}\t Cols: {}\t NaNs: {}".format(d, 
                                                    df_data[d].shape[0], 
                                                    df_data[d].shape[1], 
                                                    sum(df_data[d].isnull().sum())))
    # Pass hub data to cleaning function
    df_data[d] = clean_df(df_data[d],df_hubs)
    check_data(df_data[d])

Cleaning the data:
Month: 2017_07 
Rows: 565142	 Cols: 12	 NaNs: 358366
Drop lat/lon rows: 0
Trivial dur: 14746 dist: 0
Outlier loc: 0 dur: 0 dist: 0
Rows: 550396	 Cols: 13	 NaNs: 0
Month: 2017_08 
Rows: 557639	 Cols: 12	 NaNs: 291943
Drop lat/lon rows: 0
Trivial dur: 30863 dist: 0
Outlier loc: 0 dur: 0 dist: 0
Rows: 541522	 Cols: 13	 NaNs: 0
Month: 2017_09 
Rows: 485489	 Cols: 12	 NaNs: 232319
Drop lat/lon rows: 0
Trivial dur: 46848 dist: 0
Outlier loc: 0 dur: 0 dist: 0
Rows: 469504	 Cols: 13	 NaNs: 0
Month: 2017_Q4 
Rows: 669239	 Cols: 12	 NaNs: 157407
Drop lat/lon rows: 0
Trivial dur: 78308 dist: 0
Outlier loc: 0 dur: 0 dist: 0
Rows: 637779	 Cols: 13	 NaNs: 0
Month: 2018_Q1 
Rows: 387145	 Cols: 12	 NaNs: 44815
Drop lat/lon rows: 62
Trivial dur: 99494 dist: 0
Outlier loc: 0 dur: 116 dist: 0
Rows: 365781	 Cols: 13	 NaNs: 0
Month: 2018_Q2 
Rows: 1059681	 Cols: 12	 NaNs: 391775
Drop lat/lon rows: 145
Trivial dur: 135340 dist: 0
Outlier loc: 0 dur: 536 dist: 0
Rows: 1023270	 Cols: 13	 Na

<hr>
<a name="merge"></a>

## <span style='color:#3b748a'> VI. Merge the DataFrames into 1 big DataFrame</span>

In [25]:
n_rows = 0
df = pd.DataFrame()
for d in trip_data:
    n_rows += df_data[d].shape[0]
    df = df.append(df_data[d])

if n_rows != df.shape[0]:
    print("There is a problem with the DataFrame merge!")

<hr>
<a name="explore"></a>

## <span style='color:#3b748a'> VII. Explore the data.</span>

In [26]:
df.head()    

,Start Hub,Start Latitude,Start Longitude,Start Date,Start Time,End Hub,End Latitude,End Longitude,End Date,End Time,Bike Name,Distance [Miles],Duration
0,Clinton St & Roosevelt Rd,41.867118,-87.641088,2017-07-31,2017-07-31 23:59:43,Halsted St & 18th St,41.857499,-87.646277,2017-08-01,2017-08-01 00:09:28,3121,0.715798,00:09:45
1,Broadway & Ridge Ave,41.984045,-87.660274,2017-07-31,2017-07-31 23:59:34,Broadway & Wilson Ave,41.965221,-87.658139,2017-08-01,2017-08-01 00:10:04,6062,1.303801,00:10:30
2,Clinton St & Roosevelt Rd,41.867118,-87.641088,2017-07-31,2017-07-31 23:59:27,Halsted St & 18th St,41.857499,-87.646277,2017-08-01,2017-08-01 00:09:24,2659,0.715798,00:09:57
3,Wells St & Concord Ln,41.912133,-87.634656,2017-07-31,2017-07-31 23:59:27,Wells St & Evergreen Ave,41.906724,-87.634830,2017-08-01,2017-08-01 00:06:59,4230,0.373419,00:07:32
4,Larrabee St & Webster Ave,41.921822,-87.644140,2017-07-31,2017-07-31 23:59:18,California Ave & Francis Pl,41.918121,-87.696972,2017-08-01,2017-08-01 00:18:28,5415,2.735206,00:19:10


In [27]:
df.shape

(3588252, 13)

In [28]:
df.columns

Index(['Start Hub', 'Start Latitude', 'Start Longitude', 'Start Date',
       'Start Time', 'End Hub', 'End Latitude', 'End Longitude', 'End Date',
       'End Time', 'Bike Name', 'Distance [Miles]', 'Duration'],
      dtype='object')

In [29]:
df.describe()

,Start Latitude,Start Longitude,End Latitude,End Longitude,Bike Name,Distance [Miles],Duration
count,3.588252e+06,3.588252e+06,3.588252e+06,3.588252e+06,3.588252e+06,3.588252e+06,3588252
mean,4.189866e+01,-8.764158e+01,4.189922e+01,-8.764186e+01,3.584663e+03,1.288617e+00,0 days 00:17:06.724361
std,3.672525e-02,2.254192e-02,3.692653e-02,2.263170e-02,1.936490e+03,1.007049e+00,0 days 00:28:37.601785
min,4.173665e+01,-8.780391e+01,4.173665e+01,-8.780391e+01,1.000000e+00,0.000000e+00,0 days 00:03:00
25%,4.188042e+01,-8.765304e+01,4.188042e+01,-8.765345e+01,1.886000e+03,6.176514e-01,0 days 00:07:04
50%,4.189180e+01,-8.763851e+01,4.189228e+01,-8.763868e+01,3.745000e+03,9.935970e-01,0 days 00:11:43
75%,4.191718e+01,-8.762603e+01,4.191831e+01,-8.762618e+01,5.345000e+03,1.675962e+00,0 days 00:19:46
max,4.206431e+01,-8.754939e+01,4.206431e+01,-8.754939e+01,6.471000e+03,1.843041e+01,0 days 23:57:04


In [30]:
if False:
    check_data(df)
    check_cols(df)

<hr>
<a name="write"></a>

## <span style='color:#3b748a'>VII. Write the full DataFrame to a csv file.</span>

In [31]:
df.to_csv('../data/chi/trips_all.csv', index=False)